## Webscraping FIIs

Site referência: https://www.fundsexplorer.com.br/ranking

In [ ]:
# !pip install -q -U watermark 

In [1]:
import pandas as pd
import json
import requests as r
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

#### Versões utilizadas

In [2]:
%reload_ext watermark
%watermark --iversions

requests: 2.32.3
pandas  : 2.2.3
bs4     : 4.13.3
json    : 2.0.9



In [3]:
url = "https://www.fundsexplorer.com.br/wp-json/funds/v1/get-ranking"

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cookie": "suno_checkout_userid=ad336406-522a-42e8-ac91-349945b25c8a; _gcl_au=1.1.2079664177.1741828992; __gtm_referrer=https%3A%2F%2Fwww.google.com%2F; _clck=1ot281v%7C2%7Cfu6%7C0%7C1898; _hjSession_1183503=eyJpZCI6IjE4YjkxMmFjLTIxNGItNGZlNC05NGFlLTE5YmMwODZmNzA5MiIsImMiOjE3NDE4Mjg5OTMyNDcsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MH0=; _ga=GA1.1.630256352.1741828993; FPAU=1.1.2079664177.1741828992; __hstc=173632188.cd3633df4a95797d9d484304826184c0.1741828994294.1741828994294.1741828994294.1; hubspotutk=cd3633df4a95797d9d484304826184c0; __hssrc=1; hotid=eyJjaWQiOiIxNzMyNjQ5NzkxODA0MzY3MDk3MTk2ODE4MzQ3ODQwIiwiYmlkIjoiMTczMjY0OTc5MTgwNDM2NzA5NzE5NjgxODM0Nzg0MCIsInNpZCI6ImVmNTZlNzQ0MTRkYTQ5NDRiOTI3Y2I3NmZjNGEzZjRkIn0=; wisepops_visits=%5B%222025-03-13T01%3A23%3A12.327Z%22%5D; _fbp=fb.2.1741828995015.675119474247774340; messagesUtk=cb068ce4c95944d28cd71ef425d1f3ae; hs-messages-hide-welcome-message=true; FPGSID=1.1741828994.1741829763.G-VN673HEDSJ.cBnUWwf23EDte3Fu41fcVQ; cf_clearance=ovlcDHQmQQwMnCO6MYDq9tfCCCtro5PlVNXDTo5rrVM-1741829764-1.2.1.1-JHdUrEIU2rbFSDmk00ws4TyYnX0TulgvETPCyo1.wJd5Mn2YbJfIO2M6CKzbCvc_UGKblzViUaamovWVUkweH28HmjAVDFGLAgb3zsa2Fq..5l1V7BtkuGy838swD4JcySezQ_uXw7I7EHeJa6B3hL5XjvfrQap9WfeHp45S.LVyB5D7efwVNtvRNLXJu0YCXNf_uwEPsjNJ2A0UrV2_q3jFsF7Cbk72eTFPeOZ.Uq6Rox3afoauE31KJG2bcFbZZVCVkSQA7Nd3ieYh4Lbwudnd32kIPrWsWJsltSscHCcpZbvTJ2UU5hgH.KrLA3n9aYntEGGlVIIgSHFJeAcPQUmwOe6oy4xvkafcQeS2Ar4; _hjSessionUser_1183503=eyJpZCI6IjEyOTFhYmNjLTk3ZmEtNTVkNy1hMmY2LWNlYmJlODI1ZThkNSIsImNyZWF0ZWQiOjE3NDE4Mjg5OTMyNDYsImV4aXN0aW5nIjp0cnVlfQ==; wisepops=%7B%22popups%22%3A%7B%7D%2C%22sub%22%3A0%2C%22ucrn%22%3A2%2C%22cid%22%3A%2252100%22%2C%22v%22%3A5%2C%22bandit%22%3A%7B%22recos%22%3A%7B%7D%7D%7D; _clsk=1w19r3t%7C1741829776510%7C4%7C1%7Cf.clarity.ms%2Fcollect; __hssc=173632188.3.1741828994294; wisepops_visitor=%7B%22Uiy2Q6Vhk4%22%3A%22079c853a-e2a4-4608-8104-1a4d7d24a124%22%7D; wisepops_session=%7B%22arrivalOnSite%22%3A%222025-03-13T01%3A23%3A12.327Z%22%2C%22mtime%22%3A1741829776740%2C%22pageviews%22%3A3%2C%22popups%22%3A%7B%7D%2C%22bars%22%3A%7B%7D%2C%22embeds%22%3A%7B%7D%2C%22sticky%22%3A%7B%7D%2C%22countdowns%22%3A%7B%7D%2C%22src%22%3A%22https%3A%2F%2Fwww.google.com%2F%22%2C%22utm%22%3A%7B%7D%2C%22testIp%22%3Anull%7D; _ga_VN673HEDSJ=GS1.1.1741828993.1.1.1741829790.0.0.1432253131",
    "priority": "u=1, i",
    "referer": "https://www.fundsexplorer.com.br/ranking",
    "sec-ch-ua": '"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "x-funds-nonce": "61495f60b533cc40ad822e054998a3190ea9bca0d94791a1da"
}

response = r.get(url, headers=headers).json()
response = json.loads(response)
df_fiis = pd.DataFrame(response)

In [4]:
df_fiis.head()

,post_id,ticker,dividendo,yeld,media_yield_3m,soma_yield_3m,media_yield_6m,soma_yield_6m,media_yield_12m,soma_yield_12m,...,patrimonio,pvp,p_vpa,post_title,ativos,volatility,numero_cotista,tx_gestao,tx_admin,tx_performance
0,2555,AAGR11,1.2088,1.2500,1.2900,3.8700,1.2200,7.3200,1.1992,14.3900,...,0,0,NaN,AAGR11,,67.033057810883,0,,,
1,495,AAZQ11,0.0950,1.4400,1.5800,4.7400,1.4700,8.8200,1.3917,16.7000,...,207210754.51,0.8445,0.844548,AAZQ11,,22.684753722077,29667,,,
2,238,ABCP11,0.5000,0.6700,0.7067,2.1200,0.8967,5.3800,0.8742,10.4900,...,506969232.82,0.7142,0.714193,ABCP11,1,33.154934803299,14987,,,
3,177,AFHI11,0.9800,1.0400,1.0667,3.2000,1.0483,6.2900,1.0392,12.4700,...,424611949.75,0.9999,0.999893,AFHI11,14,11.760353259438,42586,,,
4,553,AGRX11,0.1100,1.3800,1.3833,4.1500,1.2050,7.2300,1.2633,15.1600,...,183189143.82,0.7859,0.785924,AGRX11,0,32.527217888726,19514,,,


In [5]:
df_fiis.tail()

,post_id,ticker,dividendo,yeld,media_yield_3m,soma_yield_3m,media_yield_6m,soma_yield_6m,media_yield_12m,soma_yield_12m,...,patrimonio,pvp,p_vpa,post_title,ativos,volatility,numero_cotista,tx_gestao,tx_admin,tx_performance
521,537,YUFI11B,0.6331,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,NaN,YUFI11B,,,0,,,
522,3486,ZAGH11,0.0700,0.7800,0.5467,1.6400,0.5350,3.2100,0.5017,6.0200,...,79705199.51,1.0379,NaN,ZAGH11,1,967.09723554645,400,,,
523,2972,ZAVC11,0.1300,0.0000,0.9200,2.7600,1.1400,6.8400,1.2850,12.8500,...,31579152.62,0.8961,NaN,ZAVC11,1,71.24569506236,160,,,
524,1866,ZAVI11,1.0500,1.0600,1.2600,3.7800,1.2133,7.2800,1.1367,13.6400,...,155146750.77,0.7235,NaN,ZAVI11,10,41.445976464842,3353,,,
525,295,ZIFI11,3.3050,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,83903104.89,0.3396,NaN,ZIFI11,14,58.00348306158,98,,,


In [6]:
df_fiis.to_csv('datasets/df_fiis_completo.csv', index=False)

In [7]:
df_fiis = df_fiis[['ticker',
                   'setor',
                   'dividendo',
                   'liquidezmediadiaria',
                   'pvp',
                   'yeld',
                   'media_yield_12m',
                   'soma_yield_12m',
                   'soma_yield_ano_corrente',
                   'rentabilidade',
                   'rentabilidade_mes',
                   'variacao_cotacao_mes',
                   'cotacao_fechamento',
                   'vpa',
                   'numero_cotista',
                   'volatility']]

df_fiis.columns = df_fiis.columns.str.upper()

In [8]:
df_fiis

,TICKER,SETOR,DIVIDENDO,LIQUIDEZMEDIADIARIA,PVP,YELD,MEDIA_YIELD_12M,SOMA_YIELD_12M,SOMA_YIELD_ANO_CORRENTE,RENTABILIDADE,RENTABILIDADE_MES,VARIACAO_COTACAO_MES,COTACAO_FECHAMENTO,VPA,NUMERO_COTISTA,VOLATILITY
0,AAGR11,Indefinido,1.2088,8166.76,0,1.2500,1.1992,14.3900,3.8700,4.0810,10.2551,8.8939,96.9700,0.0000,0,67.033057810883
1,AAZQ11,Indefinido,0.0950,548840.62,0.8445,1.4400,1.3917,16.7000,4.7400,-1.1578,-1.0997,-2.5037,6.6200,8.6200,29667,22.684753722077
2,ABCP11,Shoppings,0.5000,64576.19,0.7142,0.6700,0.8742,10.4900,2.1200,14.5576,2.5183,1.8360,74.8800,107.6600,14987,33.154934803299
3,AFHI11,Papéis,0.9800,802593.9,0.9999,1.0400,1.0392,12.4700,3.2000,11.8905,7.8256,6.7157,94.2300,93.2100,42586,11.760353259438
4,AGRX11,Indefinido,0.1100,154992.76,0.7859,1.3800,1.2633,15.1600,4.1500,12.0723,7.0122,5.5556,7.9800,10.2300,19514,32.527217888726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,YUFI11B,Indefinido,0.6331,,0,0.0000,0.0000,0.0000,None,0.0000,0.0000,0.0000,0.0000,105.5322,0,
522,ZAGH11,Serviços Financeiros Diversos,0.0700,5236,1.0379,0.7800,0.5017,6.0200,2.1000,-28.0877,-12.1955,-12.8751,9.0000,None,400,967.09723554645
523,ZAVC11,Indefinido,0.1300,8738.24,0.8961,0.0000,1.2850,12.8500,0.0000,None,0.0000,0.0000,None,None,160,71.24569506236
524,ZAVI11,Indefinido,1.0500,230816.71,0.7235,1.0600,1.1367,13.6400,3.7800,4.8175,19.2798,18.0287,99.5100,None,3353,41.445976464842


In [9]:
df_fiis.to_csv('datasets/df_fiis.csv', index=False)

# 📊 Dicionário de Dados - FIIs

| **Coluna**              | **Descrição** |
|-------------------------|--------------|
| `ticker`               | Código do fundo imobiliário (ex: ABCP11). |
| `setor`                | Setor do FII (ex: Shoppings, Logística, Escritórios). |
| `dividendo`            | Valor do último dividendo pago. |
| `liquidezmediadiaria`  | Média de negociação diária do fundo. |
| `pvp`                  | Preço sobre Valor Patrimonial (P/VP). Indica se a cota está barata ou cara em relação ao patrimônio. |
| `yeld`                 | Dividend Yield mais recente (%). |
| `media_yield_12m`      | Média do Dividend Yield nos últimos 12 meses. |
| `soma_yield_12m`       | Soma dos dividendos pagos nos últimos 12 meses. |
| `soma_yield_ano_corrente` | Soma dos dividendos pagos no ano atual. |
| `rentabilidade`        | Rentabilidade total considerando dividendos e valorização da cota. |
| `rentabilidade_mes`    | Rentabilidade no último mês. |
| `variacao_cotacao_mes` | Variação percentual do preço da cota no último mês. |
| `cotacao_fechamento`   | Último preço de fechamento da cota. |
| `vpa`                  | Valor Patrimonial da Cota (VPA). |
| `numero_cotista`       | Quantidade de cotistas do fundo. |
| `volatility`           | Volatilidade do preço da cota. |

---

**💡 Notas:**  
- O **P/VP** indica se o fundo está sobrevalorizado (>1) ou subvalorizado (<1).  
- A **liquidez média diária** mostra se o FII tem fácil negociação no mercado.  
- O **Dividend Yield** (Yeld) ajuda a identificar fundos com bons pagamentos periódicos.  
- A **volatilidade** pode ser usada para medir o risco de oscilação do fundo.  

In [10]:
df_fiis.SETOR.value_counts()

SETOR
Indefinido                          124
Papéis                               96
Lajes Corporativas                   61
Misto                                44
Imóveis Industriais e Logísticos     44
Fundo de Desenvolvimento             35
Fundo de Fundos                      35
Shoppings                            29
Imóveis Residenciais                 10
Imóveis Comerciais - Outros           7
Hospitalar                            6
Varejo                                5
Educacional                           5
Agências de Bancos                    5
Serviços Financeiros Diversos         4
Hotéis                                4
Outros                                4
Logística                             2
Incorporações                         2
                                      1
Fiagro                                1
Agricultura                           1
Tecidos. Vestuário e Calçados         1
Name: count, dtype: int64

In [11]:
# Valores nulos
df_fiis.isna().sum()

TICKER                       0
SETOR                        0
DIVIDENDO                    0
LIQUIDEZMEDIADIARIA          0
PVP                          0
YELD                         3
MEDIA_YIELD_12M              3
SOMA_YIELD_12M               3
SOMA_YIELD_ANO_CORRENTE      4
RENTABILIDADE               99
RENTABILIDADE_MES            3
VARIACAO_COTACAO_MES         3
COTACAO_FECHAMENTO          84
VPA                        213
NUMERO_COTISTA               0
VOLATILITY                   1
dtype: int64

In [12]:
def get_fii_properties(ticker):
    url = f"https://www.fundsexplorer.com.br/wp-json/funds/v1/get-all-properties?ticker={ticker}"
    
    response = r.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data if data else "Nenhuma informação encontrada."
    return f"Erro {response.status_code} ao acessar a API."

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "cookie": "suno_checkout_userid=ad336406-522a-42e8-ac91-349945b25c8a; _gcl_au=1.1.2079664177.1741828992; __gtm_referrer=https%3A%2F%2Fwww.google.com%2F; _clck=1ot281v%7C2%7Cfu6%7C0%7C1898; _hjSession_1183503=eyJpZCI6IjE4YjkxMmFjLTIxNGItNGZlNC05NGFlLTE5YmMwODZmNzA5MiIsImMiOjE3NDE4Mjg5OTMyNDcsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MH0=; _ga=GA1.1.630256352.1741828993; FPAU=1.1.2079664177.1741828992; __hstc=173632188.cd3633df4a95797d9d484304826184c0.1741828994294.1741828994294.1741828994294.1; hubspotutk=cd3633df4a95797d9d484304826184c0; __hssrc=1; hotid=eyJjaWQiOiIxNzMyNjQ5NzkxODA0MzY3MDk3MTk2ODE4MzQ3ODQwIiwiYmlkIjoiMTczMjY0OTc5MTgwNDM2NzA5NzE5NjgxODM0Nzg0MCIsInNpZCI6ImVmNTZlNzQ0MTRkYTQ5NDRiOTI3Y2I3NmZjNGEzZjRkIn0=; wisepops_visits=%5B%222025-03-13T01%3A23%3A12.327Z%22%5D; _fbp=fb.2.1741828995015.675119474247774340; messagesUtk=cb068ce4c95944d28cd71ef425d1f3ae; hs-messages-hide-welcome-message=true; FPGSID=1.1741828994.1741829763.G-VN673HEDSJ.cBnUWwf23EDte3Fu41fcVQ; cf_clearance=ovlcDHQmQQwMnCO6MYDq9tfCCCtro5PlVNXDTo5rrVM-1741829764-1.2.1.1-JHdUrEIU2rbFSDmk00ws4TyYnX0TulgvETPCyo1.wJd5Mn2YbJfIO2M6CKzbCvc_UGKblzViUaamovWVUkweH28HmjAVDFGLAgb3zsa2Fq..5l1V7BtkuGy838swD4JcySezQ_uXw7I7EHeJa6B3hL5XjvfrQap9WfeHp45S.LVyB5D7efwVNtvRNLXJu0YCXNf_uwEPsjNJ2A0UrV2_q3jFsF7Cbk72eTFPeOZ.Uq6Rox3afoauE31KJG2bcFbZZVCVkSQA7Nd3ieYh4Lbwudnd32kIPrWsWJsltSscHCcpZbvTJ2UU5hgH.KrLA3n9aYntEGGlVIIgSHFJeAcPQUmwOe6oy4xvkafcQeS2Ar4; _hjSessionUser_1183503=eyJpZCI6IjEyOTFhYmNjLTk3ZmEtNTVkNy1hMmY2LWNlYmJlODI1ZThkNSIsImNyZWF0ZWQiOjE3NDE4Mjg5OTMyNDYsImV4aXN0aW5nIjp0cnVlfQ==; wisepops=%7B%22popups%22%3A%7B%7D%2C%22sub%22%3A0%2C%22ucrn%22%3A2%2C%22cid%22%3A%2252100%22%2C%22v%22%3A5%2C%22bandit%22%3A%7B%22recos%22%3A%7B%7D%7D%7D; _clsk=1w19r3t%7C1741829776510%7C4%7C1%7Cf.clarity.ms%2Fcollect; __hssc=173632188.3.1741828994294; wisepops_visitor=%7B%22Uiy2Q6Vhk4%22%3A%22079c853a-e2a4-4608-8104-1a4d7d24a124%22%7D; wisepops_session=%7B%22arrivalOnSite%22%3A%222025-03-13T01%3A23%3A12.327Z%22%2C%22mtime%22%3A1741829776740%2C%22pageviews%22%3A3%2C%22popups%22%3A%7B%7D%2C%22bars%22%3A%7B%7D%2C%22embeds%22%3A%7B%7D%2C%22sticky%22%3A%7B%7D%2C%22countdowns%22%3A%7B%7D%2C%22src%22%3A%22https%3A%2F%2Fwww.google.com%2F%22%2C%22utm%22%3A%7B%7D%2C%22testIp%22%3Anull%7D; _ga_VN673HEDSJ=GS1.1.1741828993.1.1.1741829790.0.0.1432253131",
    "priority": "u=1, i",
    "referer": "https://www.fundsexplorer.com.br/ranking",
    "sec-ch-ua": '"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "x-funds-nonce": "61495f60b533cc40ad822e054998a3190ea9bca0d94791a1da"
}

ticker = "SNFF11"
properties = get_fii_properties(ticker)
properties = json.loads(properties)
df = pd.DataFrame(properties)

In [13]:
df.head(3)

,id,ticker,created_at,updated_at,nome,endereco,endereco_completo,bairro,cidade,estado,latitude,longitude,area,vacancia,inadimplencia,objetivo,percentual_sobre_receita
0,13374,SNFF11,2024-09-27 15:38:38,2024-09-27 15:38:38,Nova Andradina - MS (Mat. 89),Rua Milton Modesto,"Rua Milton Modesto, nº 1.503, Cento, Nova Andr...",Nova Andradina,Nova Andradina - MS,MS,-22.2390200000,-53.3393300000,1173.0000,None,None,Renda,17.6200
1,13375,SNFF11,2024-09-27 15:38:38,2024-09-27 15:38:38,Sapezal - MT (Mat. 411),Rua dos Cascudo,"Rua do Cascudo, nº 1220, Centro, Sapezal/MT",Sapezal,Sapezal - MT,MT,-13.5436700000,-58.8241200000,512.0000,None,None,Renda,11.3200
2,13376,SNFF11,2024-09-27 15:38:38,2024-09-27 15:38:38,Rio Branco - AC (Mat. 14.797),Rua Terra,"Estrada da Floresta, nº 3811, Bairro Floresta ...",Nova Esperança,Rio Branco - AC,AC,-9.9768100000,-67.8454600000,5000.0000,None,None,Renda,23.2800


In [14]:
df = df[['id',
         'ticker',
         'estado',
         'area',
         'percentual_sobre_receita']]

In [15]:
df

,id,ticker,estado,area,percentual_sobre_receita
0,13374,SNFF11,MS,1173.0000,17.6200
1,13375,SNFF11,MT,512.0000,11.3200
2,13376,SNFF11,AC,5000.0000,23.2800
3,13377,SNFF11,MS,600.0000,4.7100
4,13378,SNFF11,MS,930.0000,12.0600
5,13379,SNFF11,MT,450.0000,5.2900
6,13380,SNFF11,MS,660.0000,6.7200
7,13381,SNFF11,RO,729.0000,8.0900
8,13382,SNFF11,AC,406.0000,3.3300
9,13383,SNFF11,MS,910.3000,7.5600


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        12 non-null     object
 1   ticker                    12 non-null     object
 2   estado                    12 non-null     object
 3   area                      12 non-null     object
 4   percentual_sobre_receita  10 non-null     object
dtypes: object(5)
memory usage: 608.0+ bytes


In [17]:
df['percentual_sobre_receita'] = pd.to_numeric(df['percentual_sobre_receita'], errors='coerce')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        12 non-null     object 
 1   ticker                    12 non-null     object 
 2   estado                    12 non-null     object 
 3   area                      12 non-null     object 
 4   percentual_sobre_receita  10 non-null     float64
dtypes: float64(1), object(4)
memory usage: 608.0+ bytes


In [19]:
df.groupby('estado')['percentual_sobre_receita']\
      .agg(['count', 'mean', 'min', 'max', 'sum'])\
      .reset_index()\
      .sort_values(by='sum', ascending=False)

,estado,count,mean,min,max,sum
1,MS,5,9.734,4.71,17.62,48.67
0,AC,2,13.305,3.33,23.28,26.61
2,MT,2,8.305,5.29,11.32,16.61
3,RO,1,8.090,8.09,8.09,8.09


In [20]:
df.groupby('area')['percentual_sobre_receita']\
    .agg(['sum'])\
    .reset_index()\
    .sort_values(by='sum', ascending=False)

,area,sum
4,5000.0000,23.28
0,1173.0000,17.62
11,930.0000,12.06
5,512.0000,11.32
9,729.0000,8.09
10,910.3000,7.56
8,660.0000,6.72
3,450.0000,5.29
6,600.0000,4.71
2,406.0000,3.33


In [21]:
df['percentual_sobre_receita'].sum()

np.float64(99.98)